<a href="https://colab.research.google.com/github/Harishcj7/CODSOFT/blob/main/GenAI_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q PyMuPDF faiss-cpu sentence-transformers langchain
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from google.colab import files
uploaded = files.upload()

Saving sample.pdf.pdf to sample.pdf.pdf


In [10]:
import os

# Replace with your actual Groq API key
os.environ["GROQ_API_KEY"] = "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"  # <-- Paste your key here

In [11]:
import fitz  # PyMuPDF
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from groq import Groq  # Groq SDK

# 1. PDF Extraction
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# 2. Chunking
def chunk_text(text, chunk_size=500, chunk_overlap=100):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_text(text)

# 3. Embedding
embedder = SentenceTransformer('all-MiniLM-L6-v2')

def get_embeddings(chunks):
    return embedder.encode(chunks, convert_to_tensor=False)

# 4. Store in FAISS
def store_in_faiss(embeddings):
    dim = embeddings[0].shape[0]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))
    return index

# 5. Semantic Search
def search(query, index, embedder, chunks, top_k=5):
    query_vec = embedder.encode([query])
    distances, indices = index.search(np.array(query_vec), top_k)
    return [chunks[i] for i in indices[0]]

# 6. Groq LLM for Answering
def generate_answer(query, context_chunks):
    context = "\n".join(context_chunks)
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"

    client = Groq(api_key=os.getenv("GROQ_API_KEY"))
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [14]:
# Set filename
pdf_file = list(uploaded.keys())[0]
query = input("Enter your question: ")

# RAG flow
text = extract_text_from_pdf(pdf_file)
chunks = chunk_text(text)
embeddings = get_embeddings(chunks)
index = store_in_faiss(embeddings)
relevant_chunks = search(query, index, embedder, chunks)
answer = generate_answer(query, relevant_chunks)

print("\n🧠 Answer:\n", answer)

Enter your question: How many schengen countries are listed in the pdf?

🧠 Answer:
 Based on the provided text, there are 26 Schengen countries listed:

1. Austria
2. Belgium
3. Croatia
4. Czech Republic
5. Denmark
6. Estonia
7. Finland
8. France
9. Germany
10. Greece
11. Hungary
12. Iceland
13. Italy
14. Latvia
15. Lithuania
16. Luxembourg
17. Malta
18. Netherlands
19. Norway
20. Poland
21. Portugal
22. Slovakia
23. Slovenia
24. Spain
25. Sweden
26. Switzerland
